# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data=pd.read_csv('weather_csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.5906,26.8910,58.15,68,17,16.78,ZA,1621203687
1,Mossamedes,-15.1961,12.1522,74.52,57,0,5.39,AO,1621203687
2,Torbay,47.6666,-52.7314,41.00,75,20,12.66,CA,1621203687
3,Atuona,-9.8000,-139.0333,79.25,68,10,14.54,PF,1621203688
4,Yangi Marg`ilon,40.4272,71.7189,66.20,34,80,9.22,UZ,1621203688


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weather_data["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_data[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_weather_df = weather_data.loc[(weather_data['Max Temp'] < 80 )&(weather_data['Max Temp']> 70)&(weather_data['Cloudiness'] ==0)&(weather_data['Wind Speed']<10),: ]
new_weather_df.reset_index(inplace=True)
new_weather_df.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,Mossamedes,-15.1961,12.1522,74.52,57,0,5.39,AO,1621203687
1,32,Saint-Pierre,-21.3393,55.4781,71.60,60,0,9.22,RE,1621203696
2,102,Saint-Philippe,-21.3585,55.7679,73.40,60,0,9.22,RE,1621203714
3,133,Touros,-5.1989,-35.4608,77.00,88,0,6.91,BR,1621203722
4,275,San Lorenzo,-25.3333,-57.5333,78.80,61,0,6.33,PY,1621203765


In [6]:
del new_weather_df['index']

In [7]:
new_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.1961,12.1522,74.52,57,0,5.39,AO,1621203687
1,Saint-Pierre,-21.3393,55.4781,71.60,60,0,9.22,RE,1621203696
2,Saint-Philippe,-21.3585,55.7679,73.40,60,0,9.22,RE,1621203714
3,Touros,-5.1989,-35.4608,77.00,88,0,6.91,BR,1621203722
4,San Lorenzo,-25.3333,-57.5333,78.80,61,0,6.33,PY,1621203765


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = new_weather_df
hotel_df['Hotel_name']=''
hotel_df.head()

C:\Users\jacke\Downloads\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name
0,Mossamedes,-15.1961,12.1522,74.52,57,0,5.39,AO,1621203687,
1,Saint-Pierre,-21.3393,55.4781,71.60,60,0,9.22,RE,1621203696,
2,Saint-Philippe,-21.3585,55.7679,73.40,60,0,9.22,RE,1621203714,
3,Touros,-5.1989,-35.4608,77.00,88,0,6.91,BR,1621203722,
4,San Lorenzo,-25.3333,-57.5333,78.80,61,0,6.33,PY,1621203765,


In [9]:
hotel = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Lat']
    long = new_weather_df.loc[i]['Lng']

# set up a parameters dictionary
    params = {
    "location": f'{lat},{long}',
    "radius": 5000,
    "type": "hotel",
    "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_weather= response.json()
    
    try:
        hotel.append(response_weather['results'][0]['name'])
    except:
        hotel.append("")
hotel_df["Hotel_Name"] = hotel
hotel_df = hotel_df.dropna(how='any')

hotel_df.head()

        




C:\Users\jacke\Downloads\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name,Hotel_Name
0,Mossamedes,-15.1961,12.1522,74.52,57,0,5.39,AO,1621203687,,Namibe
1,Saint-Pierre,-21.3393,55.4781,71.60,60,0,9.22,RE,1621203696,,Saint Pierre
2,Saint-Philippe,-21.3585,55.7679,73.40,60,0,9.22,RE,1621203714,,Saint-Philippe
3,Touros,-5.1989,-35.4608,77.00,88,0,6.91,BR,1621203722,,Touros
4,San Lorenzo,-25.3333,-57.5333,78.80,61,0,6.33,PY,1621203765,,San Lorenzo


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))